In [ ]:
!git clone https://github.com/ruzanovad/diploma.git

In [1]:
%ls

fromgithub.json  kaggle.ipynb  prepare_latex_vocab.ipynb


In [2]:
%cd diploma/cnn-rnn

[Errno 2] No such file or directory: 'diploma/cnn-rnn'
/home/kaiser/work/repos/diploma/cnn-rnn/notebooks


In [ ]:
%pip install evaluate
%pip install pytorch-lightning==2.0.8 

In [ ]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from model import Image2Latex, Text
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl

In [ ]:
data_path = Path('/kaggle/input/im2latex100k')
img_path = Path('/kaggle/input/im2latex100k/formula_images_processed/formula_images_processed')

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir ../tb_logs/
# %tensorboard --logdir tb_logs/ --bind_all
# %tensorboard --logdir=your_log_dir --bind_all

In [ ]:
%run main.py --batch-size 256 --accumulate-batch 512 --train --log-step 1 \
--num-workers 2 \
--max-epochs 40 --lr 0.01 --model-name conv_bilstm_lstm_train --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed \
--data-path /kaggle/input/im2latex100k \
--vocab_file data/vocab/latex_tokens.json \
--predict-img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--notebook --gpu

In [ ]:
!nvidia-smi

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
%run main.py --batch-size 4 --accumulate-batch 8 --train --log-step 1 \
--train-sample 5 --test-sample 0 --val-sample 0 --num-workers 2 \
--max-epochs 40 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k --vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--ckpt-path tb_logs/kaggle_20epoch/checkpoints/model-19-1.10.ckpt --num-layers 2

--notebook

In [ ]:
#!/bin/bash
python main.py --batch-size 128 --accumulate-batch 256 --train --log-step 64 --log-text \
--num-workers 16 --train-sample 75000 --max-time 00:06:00:00 --val-sample 8000 \
--max-epochs 20 --lr 0.01 --model-name resnet_bilstm_lstm_train --enc-type resnet_row_encoder \
--enc-dim 512 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.4 \
--decode-type beamsearch --beam-width 5 --grad-clip 1.0 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k \
--vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--gpu --ckpt-path checkpoints/model-15-0.64.ckpt  

In [1]:
%cd ..

/home/kaiser/work/repos/diploma/cnn-rnn


In [2]:
%run main.py --batch-size 8 --accumulate-batch 64 --train --log-step 64 --log-text \
--num-workers 12 --train-sample 75275 --max-time '00:04:00:00' --val-sample 8370 \
--max-epochs 20 --lr 0.01 --model-name resnet_bilstm_lstm_train --enc-type resnet_row_encoder \
--enc-dim 512 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.4 \
--decode-type beamsearch --beam-width 5 --grad-clip 1 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k --vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[INFO] Starting model from scratch.


/home/kaiser/anaconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


==========[Train]==========


/home/kaiser/anaconda3/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/kaiser/work/repos/diploma/cnn-rnn/checkpoints exists and is not empty.

  | Name      | Type             | Params | Mode  | In sizes                        | Out sizes   
--------------------------------------------------------------------------------------------------------
0 | model     | Image2Latex      | 16.8 M | train | [[1, 3, 64, 384], [1, 20], [1]] | [1, 20, 521]
1 | criterion | CrossEntropyLoss | 0      | train | ?                               | ?           
--------------------------------------------------------------------------------------------------------
16.8 M    Trainable params
0         Non-trainable params
16.8 M    Total params
67.120    Total estimated model params size (MB)
84        Modules in train mode
0         Modules in eval mode
/home/kaiser/work/repos/diploma/cnn-rnn/model/encoder.py:52: TracerWarning: Converting a tensor 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kaiser/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Truth: [\int _ { - \epsilon } ^ { \infty } d l \: \mathrm { e } ^ { - l \zeta } \int _ { - \epsilon } ^ { \infty } d l ^ { \prime } \mathrm { e } ^ { - l ^ { \prime } \zeta } l l ^ { \prime } { \frac { l ^ { \prime } - l } { l + l ^ { \prime } } } \{ 3 \, \delta ^ { \prime \prime } ( l ) - { \frac { 3 } { 4 } } t \, \delta ( l ) \} = 0 .] 
Predict: [n \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rho \rh

Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined